## Introduction.

In this notebook, you'll learn about tokens - a crucial concept tied directly to Large Language Models (LLMs).



Here's what you will learn:
- What is a token?
- Why do we use tokens?

### Why tokens (not characters or words)?

#TODO: 
- Try to explain the drawbacks of character and word approach.
- Explain that tokens are the combination of the best from both worlds.